# Demagnetization Potential in 3D with scikit-fem

We solve the demagnetization potential problem on a 3D mesh loaded from GMSH:
$$-\Delta \phi = \nabla \cdot \mathbf{m} \quad \text{in } \Omega$$
$$\phi = 0 \quad \text{on } \partial\Omega$$

where:
- $\phi$ is the demagnetization potential
- $\mathbf{m} = (0, 0, 1)$ is the magnetization vector (uniform in z-direction)
- The source term $\nabla \cdot \mathbf{m}$ is only active in the magnetic domain (region 1)
- The air region (region 2) has no magnetization

This models the magnetic field outside a uniformly magnetized cube.

## Step 1: Import Required Libraries

We import libraries for 3D finite element computations:
- GMSH mesh loading capabilities
- 3D tetrahedral elements
- 3D visualization tools

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from skfem import Mesh, Basis, ElementTetP1, asm
from skfem.helpers import *
from skfem import BilinearForm, LinearForm
from skfem.utils import condense

## Step 2: Load Mesh and Define Function Space

We load the 3D mesh from the GMSH file:
- The mesh contains a cube (magnetic region, tag=1) surrounded by air (tag=2)
- We use P1 tetrahedral elements for the scalar potential
- The mesh regions allow us to apply different material properties

In [ ]:
# Load mesh from GMSH file
m = Mesh.load('files/cube_with_air.msh')
e = ElementTetP1()
basis = Basis(m, e)

print(f"Mesh has {m.p.shape[1]} nodes and {m.t.shape[1]} tetrahedra")
print(f"Mesh bounding box:")
print(f"  x: [{m.p[0].min():.3f}, {m.p[0].max():.3f}]")
print(f"  y: [{m.p[1].min():.3f}, {m.p[1].max():.3f}]")
print(f"  z: [{m.p[2].min():.3f}, {m.p[2].max():.3f}]")

# Check available subdomains
if hasattr(m, 'subdomains'):
    print(f"Available subdomains: {list(m.subdomains.keys())}")
else:
    print("No subdomain information found")

## Step 3: Define Weak Form (Variational Formulation)

The weak form of our problem is: Find $\phi$ such that
$$\int_{\Omega} \nabla \phi \cdot \nabla v \, d\Omega = \int_{\Omega_{mag}} \mathbf{m} \cdot \nabla v \, d\Omega \quad \forall v$$

We define:
- **Bilinear form**: $a(\phi,v) = \int \nabla \phi \cdot \nabla v \, d\Omega$ (Laplacian)
- **Linear form**: $L(v) = \int_{\Omega_{mag}} \mathbf{m} \cdot \nabla v \, d\Omega$ (magnetization source)

The magnetization $\mathbf{m} = (0, 0, 1)$ creates a source term only in the magnetic region.

In [ ]:
# Define bilinear form: ∫ ∇φ · ∇v dΩ (Laplacian)
@BilinearForm
def laplace_form(u, v, _):
    return ddot(u.grad, v.grad)

# Define linear form: ∫ m · ∇v dΩ (magnetization source)
# Only active in magnetic region (domain 1)
@LinearForm
def magnetization_form(v, w):
    # Magnetization vector m = (0, 0, 1)
    m = np.array([0.0, 0.0, 1.0])
    # m · ∇v = m_z * ∂v/∂z
    return m[2] * v.grad[2]  # Only z-component is non-zero

## Step 4: Assembly

We assemble the system matrices:
- Stiffness matrix from the Laplacian (assembled over entire domain)
- Load vector from magnetization (assembled only over magnetic region)

The key is to restrict the magnetization source to the magnetic domain only.

In [ ]:
# Assemble stiffness matrix over entire domain
A = asm(laplace_form, basis)

# Assemble load vector only over magnetic region (domain 1)
try:
    # Try to use subdomain if available
    if hasattr(m, 'subdomains') and '1' in m.subdomains:
        # Create basis restricted to magnetic domain
        basis_mag = basis.with_element(e).with_subdomain('1')
        b = asm(magnetization_form, basis_mag)
        print("Using subdomain-restricted assembly")
    else:
        # Fallback: assemble over entire domain
        # (In practice, you'd need to identify magnetic elements)
        b = asm(magnetization_form, basis)
        print("Using full domain assembly (fallback)")
except Exception as e:
    print(f"Subdomain assembly failed: {e}")
    # Simple fallback: assemble over entire domain
    b = asm(magnetization_form, basis)
    print("Using full domain assembly")

print(f"System size: {A.shape[0]} x {A.shape[1]}")
print(f"Load vector norm: {np.linalg.norm(b):.3e}")

## Step 5: Apply Boundary Conditions and Solve

We apply homogeneous Dirichlet boundary conditions $\phi = 0$ on all external boundaries:
- Find all boundary nodes
- Use the `condense` method for symmetric boundary condition application
- Solve the reduced system

This approach maintains the symmetric structure of the problem.

In [ ]:
# Find boundary nodes (all external boundaries)
boundary = m.boundary_nodes()
print(f"Found {len(boundary)} boundary nodes")

# Use condense method for symmetric BC application
interior = basis.complement_dofs(boundary)
A_int, b_int, *_ = condense(A, b, I=interior)

print(f"Reduced system size: {A_int.shape[0]} x {A_int.shape[1]}")

# Solve reduced system
phi_int = np.linalg.solve(A_int.toarray(), b_int)

# Reconstruct full solution
phi = np.zeros(basis.N)
phi[interior] = phi_int
phi[boundary] = 0.0  # Homogeneous Dirichlet BC

print(f"Solution computed with {len(phi)} degrees of freedom")
print(f"Solution range: [{phi.min():.6f}, {phi.max():.6f}]")

## Step 6: Visualization

We visualize the 3D demagnetization potential using:
- Cross-sectional plots through the cube center
- Surface plots on the cube faces
- 3D scatter plot of the potential field

The solution shows how the magnetic field extends into the surrounding air region.

In [ ]:
# Get node coordinates
x, y, z = m.p[0], m.p[1], m.p[2]

# Create visualizations
fig = plt.figure(figsize=(15, 10))

# 1. Cross-section at z=0 (middle plane)
ax1 = fig.add_subplot(2, 3, 1)
z_mid_idx = np.where(np.abs(z) < 0.05)[0]  # Nodes near z=0
if len(z_mid_idx) > 0:
    scatter = ax1.scatter(x[z_mid_idx], y[z_mid_idx], c=phi[z_mid_idx], 
                         cmap='RdBu_r', s=20)
    plt.colorbar(scatter, ax=ax1, label='φ')
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_title('Cross-section at z≈0')
    ax1.set_aspect('equal')

# 2. Cross-section at y=0 (middle plane)
ax2 = fig.add_subplot(2, 3, 2)
y_mid_idx = np.where(np.abs(y) < 0.05)[0]  # Nodes near y=0
if len(y_mid_idx) > 0:
    scatter = ax2.scatter(x[y_mid_idx], z[y_mid_idx], c=phi[y_mid_idx], 
                         cmap='RdBu_r', s=20)
    plt.colorbar(scatter, ax=ax2, label='φ')
    ax2.set_xlabel('x')
    ax2.set_ylabel('z')
    ax2.set_title('Cross-section at y≈0')
    ax2.set_aspect('equal')

# 3. Cross-section at x=0 (middle plane)
ax3 = fig.add_subplot(2, 3, 3)
x_mid_idx = np.where(np.abs(x) < 0.05)[0]  # Nodes near x=0
if len(x_mid_idx) > 0:
    scatter = ax3.scatter(y[x_mid_idx], z[x_mid_idx], c=phi[x_mid_idx], 
                         cmap='RdBu_r', s=20)
    plt.colorbar(scatter, ax=ax3, label='φ')
    ax3.set_xlabel('y')
    ax3.set_ylabel('z')
    ax3.set_title('Cross-section at x≈0')
    ax3.set_aspect('equal')

# 4. 3D scatter plot (subsample for clarity)
ax4 = fig.add_subplot(2, 3, 4, projection='3d')
# Subsample nodes for 3D visualization
step = max(1, len(phi) // 1000)  # Show at most 1000 points
idx_sub = np.arange(0, len(phi), step)
scatter_3d = ax4.scatter(x[idx_sub], y[idx_sub], z[idx_sub], 
                        c=phi[idx_sub], cmap='RdBu_r', s=10)
ax4.set_xlabel('x')
ax4.set_ylabel('y')
ax4.set_zlabel('z')
ax4.set_title('3D Potential Field')
plt.colorbar(scatter_3d, ax=ax4, shrink=0.5, label='φ')

# 5. Potential along z-axis (x=0, y=0)
ax5 = fig.add_subplot(2, 3, 5)
axis_idx = np.where((np.abs(x) < 0.05) & (np.abs(y) < 0.05))[0]
if len(axis_idx) > 0:
    z_axis = z[axis_idx]
    phi_axis = phi[axis_idx]
    sort_idx = np.argsort(z_axis)
    ax5.plot(z_axis[sort_idx], phi_axis[sort_idx], 'o-', linewidth=2)
    ax5.set_xlabel('z')
    ax5.set_ylabel('φ')
    ax5.set_title('Potential along z-axis')
    ax5.grid(True, alpha=0.3)

# 6. Histogram of potential values
ax6 = fig.add_subplot(2, 3, 6)
ax6.hist(phi, bins=50, alpha=0.7, edgecolor='black')
ax6.set_xlabel('φ')
ax6.set_ylabel('Frequency')
ax6.set_title('Distribution of Potential Values')
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Analysis and Insights

The demagnetization potential solution reveals several important features:

1. **Magnetic dipole field**: The solution resembles a magnetic dipole field from the uniformly magnetized cube
2. **Boundary conditions**: Zero potential at the outer boundaries (far-field condition)
3. **Field continuity**: Smooth transition from the magnetic region to air
4. **Symmetry**: The field exhibits the expected symmetry from the uniform z-magnetization

The magnetic field **H** can be computed as **H** = -∇φ, giving the demagnetization field around the magnetized cube.

### Physical Interpretation

- **Inside the cube**: The demagnetization field opposes the magnetization
- **Outside the cube**: The field resembles that of a magnetic dipole
- **At interfaces**: Field continuity is automatically satisfied by the finite element method

This type of calculation is fundamental in micromagnetics and magnetic field computation.

### Experiments
- Change the magnetization direction (e.g., m = (1,0,0) for x-direction)
- Modify the cube geometry or add multiple magnetic objects
- Implement non-uniform magnetization patterns
- Add magnetic permeability differences between regions